In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

import pandas as pd
import xarray as xr

import sys
sys.path.append('../')

from general_purpose import cartopy_plots as cplt

In [ ]:
ts = xr.open_dataset('HEAVY--Land_and_Ocean_LatLong1.nc')

In [ ]:
ts

In [ ]:
LON,LAT = np.meshgrid(ts.longitude.data,ts.latitude.data)

## Plot the land_mask

In [ ]:
cplt.mfp(LON, LAT, ts.land_mask.data.reshape(*mean.shape,1),
         figsize=(12,6),
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=['Annual mean temperature [C]'],
        )

## Plot the climatology

In [ ]:
ts.climatology.shape

In [ ]:
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
ofl = (4,3)
cplt.mfp(LON, LAT, ts.climatology.data.transpose(1,2,0),
         figsize=(16,9),
         one_fig_layout=ofl,
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=months,
        )

### Compute climatology anomaly

In [ ]:
mean = ts.climatology.mean('month_number')

In [ ]:
cplt.mfp(LON, LAT, mean.data.reshape(*mean.shape,1),
         figsize=(12,6),
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=['Annual mean temperature [C]'],
         draw_labels=True,
         draw_gridlines=True
        )

In [ ]:
ano_clim = ts.climatology - mean

In [ ]:
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
ofl = (4,3)
cplt.mfp(LON, LAT, ano_clim.data.transpose(1,2,0),
         figsize=(16,9),
         one_fig_layout=ofl,
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=months,
        )

In [ ]:
ano_clim_periodic = np.zeros_like(ts.temperature.data)

for i in range(12):
    ano_clim_periodic[i::12] = ano_clim.data[i]

## Plot a few snapshots

In [ ]:
ts.temperature.shape

In [ ]:
i = 12*100
ofl = (4,3)
cplt.mfp(LON, LAT, ts.temperature.data[i:i+np.prod(ofl)].transpose(1,2,0),
         figsize=(16,9),
         one_fig_layout=ofl,
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=None,
        )

In [ ]:
plt.close(19)
plt.figure(figsize=(9,6), num=19)
ts.temperature.isel(latitude=125, longitude=200).plot()
plt.plot(ts.time.data, ts.temperature.isel(longitude=200, latitude=125).data - ano_clim_periodic[:,125,200])
fig.tight_layout()